* Se puden mejorar las mascaras haciendolas de un bit

In [2]:
# Image processing
import cv2
import numpy as np
import os
import glob
from ipynb.fs.full.Utils import *

In [6]:
# Para mostrar una imagen
def mostrar_img(lbl, img, reduction_ratio=0):
    mostrar_imgs([lbl],[img], reduction_ratio)

# Para mostrar varias imágenes
def mostrar_imgs(lbls, vec_img, reduction_ratio=0):
    if ((reduction_ratio >= 1) or (reduction_ratio < 0)):
        reduction_ratio = 0
    for i in range(len(vec_img)):
        h = vec_img[i].shape[0]
        h = (int)(h - (h*reduction_ratio))
        w = vec_img[i].shape[1]
        w = (int)(w - (w*reduction_ratio))
        img_rs = cv2.resize(vec_img[i], (w, h))
        cv2.imshow(lbls[i], img_rs)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
###Para escalar la imagen de 0 a max_value
def scaling(img, max_value):
    amax = np.amax(img)
    amin = np.amin(img)
    scalingIndex = max_value/(amax-amin)
    img = ((img-amin)*scalingIndex)
    img = img.astype(np.uint8)
    return img

### Devuelve el porcentaje de vegetación que hay en la imágen (255 es vegetación)
def get_total_vegetation(binary_img):
    return (np.sum(binary_img == 255) / (binary_img.shape[0] * binary_img.shape[1]))

### Busca los puntos máximos de una distribución (Buscar librerías en C++)
def get_maximum_points(img, winSize):
    #Generación de histograma
    histogram = column_histogram(img)
    #Suavizado de histograma
    min_dist = int(winSize/2)
    winMidSize = int(min_dist/2)
    smoothed = np.zeros(histogram.shape, np.int16)
    for i in range(0, len(histogram)):
        total = 0
        for j in range(0, min_dist):
            tmp = i - winMidSize + j
            if (tmp >= 0 and  tmp < len(histogram)):
                total += histogram[tmp]
        smoothed[i] = int(total/min_dist)
    #Detección de máximos locales
    indexes = []
    indexes.append(0)
    local_maximas = 1
    min_dist = winSize*1.2
    winMidSize = int(winSize/2)
    flagMeceta = False
    distance_counter = 0
    for i in range(0, len(histogram)-winSize):
        for j in range(winMidSize, winSize):
            tmp = i - winMidSize + j
            if (min_dist < tmp - indexes[local_maximas-1]):
                indexes.append(tmp)
                local_maximas += 1
            else:
                if (smoothed[tmp] > smoothed[indexes[local_maximas-1]]):
                    indexes[local_maximas-1] = tmp
                    distance_counter = 0
                    flagMeceta = False
                else:
                    if (smoothed[tmp] == smoothed[indexes[local_maximas-1]]):
                        flagMeceta = True
                        distance_counter += 1
                    else:
                        if (flagMeceta):
                            indexes[local_maximas-1] = indexes[local_maximas-1] + int((tmp - indexes[local_maximas-1])/2)
                            distance_counter = 0
                            flagMeceta = False
    #Gráfica
    indexes = np.array(indexes, np.int16)
    #------------------------#
    if (debugging):
        x = np.arange(0, img.shape[1], 1)
        fig, ax = plt.subplots()
        ax.plot(x, smoothed)
        ax.scatter(indexes, smoothed[indexes], c='blue')
        ax.grid()
        plt.show()
    #------------------------#
    return indexes

# Devuelve 1 si la posición enviada pertenece al cultivo, -1 si pertenece a la maleza
def is_crop_or_furrow_lineal(lines, p0, crop_width):
    # Crop: 1, Crop furrow: -1, None: 0¿
    if (len(lines) != 0):
        x, y = p0
        h_buffer = {}
        h_buffer[0] = lines[0].get_y(x)
        most_left = h_buffer[0] - crop_width
        most_right = h_buffer[0] + crop_width
        for line_index in range(1, len(lines)):
            h_buffer[line_index] = lines[line_index].get_y(x)
            most_left = min(most_left, h_buffer[line_index] - crop_width)
            most_right = max(most_right, h_buffer[line_index] + crop_width)
        if (y <= most_left or y >= most_right): #Caso a izquierda de primera linea o a derecha de última línea
                return 0
        for key in h_buffer.keys():  # Paso por cada curva
            c_left = h_buffer[key] - crop_width
            c_right = h_buffer[key] + crop_width
            if (y <= c_right and y >= c_left): #Si está dentro de alguna franja de cultivo, es surco
                return 1
        return -1 #Si no es ninguno de los otros, el entre surco
    else:
        return 0

# Escalar los valores para que no tenga overflow, ver como modifican los escalares
def least_squares_line(x_list, y_list, last_point):
    # Calcular el centroide en vez de usar point0
#     xc, yc = np.sum(x_list)/len(x_list), np.sum(y_list)/len(y_list)
    yc, xc = last_point
    x_list, y_list = np.array(x_list, np.uint32), np.array(y_list, np.uint32)
    sum_x, sum_y = np.sum(x_list), np.sum(y_list)
    sum_xy = np.sum(np.multiply(x_list, y_list))
    sum_x_squared = np.sum(np.multiply(x_list, x_list))
    numerator = x_list.shape[0] * sum_xy - (sum_x * sum_y)
    denominator = x_list.shape[0] * sum_x_squared - (sum_x * sum_x)
    # Si 0 es denominador, es una recta vertical, la pendiente es infinito
    m = numerator / denominator# if denominator != 0 else 999 
#     xc, y_p0 = point0
    b = int(yc - m * xc)
    return m, b  # Coeficientes: Lineal y independiente

### Promedio de una lista de valores
def mean(values_list):
    values_list = np.array(values_list, np.uint64)
    return (np.sum(values_list) / values_list.shape[0])

### Devuelve el máximo de ancho entre puntos en paralela al eje de coordenadas
def get_max_distance(indexes):
    max_distance = 0
    for i in range(1, indexes.shape[0]):
        max_distance = max(max_distance, (indexes[i] - indexes[i-1]))
    return max_distance

class Line:
    #Si la pendiente es mayor a 80, se toma como recta vertical y se debe usar x_offset
    def __init__(self, m, b):
        self.m = m
        self.b = b
        self.white_amount = -1

     # Devuelve la cantidad de vegetación (255) que hay en una recta
    def set_white_amount(self, img):
        sum = 0
        for i in range(0, img.shape[0]):
            h = self.get_y(i)
            if (h >= 0 and h < img.shape[1]):
                if (img[i, h] > 0):
                    sum = sum + 1
        self.white_amount = sum
        
    def __str__(self):
        return 'm: ' + str(self.m) + ' - b: ' + str(self.b) + ' - x_offset: ' + str(self.x_offset) + ' - white amount: ' + str(self.white_amount)
    
    # Devuelve el valor de x en una recta, dada su altura (y), si la pendiente es muy pronunciada, se toma como recta vertical
    def get_y(self, x):
#         return self.x_offset if (self.m >= 80) else int((y-self.b)/self.m)
        return int(x*self.m+self.b)
    
    def __copy__(self):
        return Line(self.m, self.b)

class MicroROI:
    def __init__(self, microimg, h_offset, v_offset):
        vert_sum, horiz_sum, _sum = 0, 0, 0
        vc, hc = int(microimg.shape[0] / 2), int(microimg.shape[1] / 2)
        for h in range(0, microimg.shape[1]):
            for v in range(0, microimg.shape[0]):
                if (microimg[v, h] > 0):  # Cambiar a binario
                    vert_sum, horiz_sum, _sum = vert_sum + v, horiz_sum + h, _sum + 1
        if (_sum != 0):
            vc, hc = int(vert_sum / _sum), int(horiz_sum / _sum)
        self.vc, self.hc = vc + v_offset, hc + h_offset
        
    def get_centroid(self):
        return self.vc, self.hc

### Devuelve la imágen con el índice seleccionado aplicado
def img_to_color_index(img, index):
    if (index == 'cive'):
        return get_CIVE(img)
    elif (index == 'exg'):
        return get_ExG(img)
    else:
        return img

def get_normalized(img):
    img = img.astype(np.uint16)
    denominator = img[:, :, 0] + img[:, :, 1] + img[:, :, 2]
    # Max 255+255+255, Min 1
    denominator = np.where(denominator == 0, 1, denominator)
    return cv2.merge((img[:, :, 0]/denominator, img[:, :, 1]/denominator, img[:, :, 2]/denominator))

def get_CIVE(img):
    img_norm = get_normalized(img)
    return 0.441*img_norm[:, :, 0]-0.811*img_norm[:, :, 1]+0.385*img_norm[:, :, 2]+18.78745

def get_ExG(img):
    r, g, b = cv2.split(img)
    r_mx = np.amax(r)
    g_mx = np.amax(g)
    b_mx = np.amax(b)
    if (r_mx == 0):
        r_mx = 1
    if (g_mx == 0):
        g_mx = 1
    if (b_mx == 0):
        b_mx = 1
    r = r / r_mx
    g = g / g_mx
    b = b / b_mx
    denominator = r + g + b
    denominator = np.where(denominator == 0, 1, denominator)
    r = r / denominator
    g = g / denominator
    b = b / denominator
    return 2*g-r-b

### Devuelve el histograma de columnas (para imágenes de 8 bits de profundidad, 255 = blanco)
def column_histogram(img_in):
    vector = np.zeros(img_in.shape[1], np.uint16)
    for col in range(0, img_in.shape[1]):
        vector[col] = np.sum(img_in[:, col] > 0)
    return vector

# Para realizar un corte en el margen superior de la imagen
def v_crop_top(img, crop_ratio):
    if (crop_ratio < 0 or crop_ratio > 1):
        crop_ratio = 0
    h2 = img.shape[0]
    h1 = (int)(h2*crop_ratio)
    w = img.shape[1]
    return img[h1:h2, 0:w]

# Variables de configuración

In [3]:
segments_amount = 20 #Cantidad de segmentos
window_ratio = 0.05 # 5% con 1024 son ventanas de 51 píxeles (ojo porque quedan pixeles a la derecha si procesar)
top_crop_ratio = 0.1
width_crop_ratio = 0.1

### Cargado de imágenes de prueba para detección de lineas de cultivo

In [5]:
#Levantar imagenes test de row_test
# img_path = "img/row_test_artificial/"
img_path = "img/consecutive/"
#img_path = "img/enhancement_test/"
images_in = []
labels = []

for file in glob.glob(img_path + '*.png'):
# for file in glob.glob(img_path + 'IMG_*.png'):
    img = cv2.imread(file, cv2.IMREAD_COLOR)
#     hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    images_in.append()
#     images_in.append(v_crop_top(hsv_enhancement(hsv), top_crop_ratio))
    labels.append(os.path.splitext(os.path.basename(file))[0])
    (os.path.basename(file))
    print(file)

img/consecutive\test26.png
img/consecutive\test27.png
img/consecutive\test28.png
img/consecutive\test29.png
img/consecutive\test30.png
img/consecutive\test31.png
img/consecutive\test32.png


### Segmentación por verde
Se puede agregar otros indices para mejorar la segmentación

Con índice de Pagola detecta, lo que a ojímetro parecen, sectores de planta con más vida

* Si se utiliza en HSV usar *segment_by_color*

In [10]:
#Segmentación por verde
seg_imgs = []

kernel_e = np.ones((3,3),np.uint8)
kernel_d = np.array(([0,1,0],[1,1,1],[0,1,0]),np.uint8)
kernel1 = np.array(([0,1,0],[1,1,1],[0,1,0]),np.uint8)
kernel2 = np.array(([1,0,1],[0,1,0],[1,0,1]),np.uint8)

for img in images_in:
    gray_bin = segment_by_color(img, binary=True)
    #Erase lo suficiente para borrar puntos, y dilatate para recuperar masa de lo mayor
#     gray_bin = cv2.erode(gray_bin, kernel_e,iterations=1)
#     gray_bin = cv2.dilate(gray_bin, kernel_d,iterations=2)
#     gray_bin = cv2.erode(gray_bin, kernel_e,iterations=1)
    seg_imgs.append(gray_bin)

In [11]:
img_show_list = []
for img in seg_imgs:
        img_show_list.append(img.astype('uint8')*255)
mostrar_imgs(labels, img_show_list)

In [12]:
#Si se usa binario, usar el box de abajo
mostrar_imgs(labels, seg_imgs)

### Operatoria para consegir lineas de cultivo (RCRD)
Hay dos posibilidades, la segunda funciona mejor cuando hay un conjunto "grande" de imágenes relacionadas
* Si se usa la primera descomentar *gray_bin = segment_by_color(img)* y comentar la siguiente
* Si se usa la segunda decomentar *gray_bin = segment_by_color_binary(img)* y comentar la anterior

In [35]:
# Sería con probabilidad de 1
seg_and = seg_imgs[0].copy()
for seg in seg_imgs:
    seg_and = cv2.bitwise_and(seg_and, seg_and, mask = seg)
#Los bordes van  a clasificarse mal si no los recorto (problema de tener fotos no consecutivas)
# cropped = h_crop(seg_and,0.1)

In [13]:
# Mediante probabilidades, aumenta el número de imágenes y aumenta la probabilidad de la fila
green_amount = 0.4
seg_and = np.zeros(seg_imgs[0].shape, np.uint8)
for seg in seg_imgs:
    seg_and = seg_and + seg
_max = (int)(np.amax(seg_and)*green_amount)
seg_and = cv2.GaussianBlur(seg_and,(7,7),cv2.BORDER_DEFAULT)
ret,seg_and = cv2.threshold(seg_and, _max,255,cv2.THRESH_BINARY)
# cropped = h_crop(seg_and,0.1)
mostrar_img('seg', seg_and)

In [48]:
#No aplicar
# kernel_d1 = np.ones((3,3), np.uint8)
# kernel_e1 = np.ones((2,2), np.uint8) 
# kernel_e2 = np.ones((5,5), np.uint8) 
# # seg_hough = cv2.dilate(seg_and, kernel, iterations=1)
# seg_hough = cv2.erode(seg_and, kernel_e1, iterations=1)
# seg_hough = cv2.dilate(seg_hough, kernel_d1, iterations=2)
# seg_hough = cv2.erode(seg_hough, kernel_e2, iterations=1)
# mostrar_img('seg', seg_hough)

### Imagen de prueba para clasificación
Se segmenta y se calculan bordes para tener referencia visual (No lo requiere el algoritmo final)

In [59]:
file_name = 'artificial_1'
original_td = cv2.imread(img_path + file_name + '.png', cv2.IMREAD_COLOR)
# original_td = h_crop(original_td, 0.1) #Para que queden de igual ancho
to_detect = cv2.cvtColor(original_td, cv2.COLOR_RGB2HSV)
to_detect = v_crop_top(segment_by_color(to_detect), top_crop_ratio)

In [60]:
kernel = np.ones((3,3), np.uint8) 
to_detect = cv2.erode(to_detect, kernel, iterations=1)

In [61]:
# mostrar_img('to_detect', to_detect)

In [63]:
to_detect_edge = cv2.Canny(to_detect,50,150)
zeros = np.zeros(to_detect.shape, np.uint8)
three_channel_edge = cv2.merge((to_detect, zeros, zeros))
mostrar_img('edges canny', three_channel_edge)

### Clasificación (FIP + RCRD)
Trabajo en segmentos, divide la imagen en 20 segmentos (se puede optar por análizar solo la parte inferior, los primeros 15 segmentos)

In [64]:
td_h, td_w = to_detect.shape
seg_height = (int)(td_h/segments_amount)
steps = (int)(td_h/seg_height) #Si td_h no es múltiplo de la cantidad de seg => steps != segments_amount
window_width = (int)(td_w*window_ratio)

#RCRD descripto como vectores para que sea más rápido verificar
offset_seg_height = 0
rcrd_vectors = np.zeros((steps, td_w), np.bool)
for i in range(0,steps):
    segment = seg_and[offset_seg_height:offset_seg_height + seg_height, 0:td_w]
    rcrd_vectors[i,:] = calculate_row(segment, seg_height, td_w)
    offset_seg_height = offset_seg_height + seg_height

In [71]:
def is_crop(window):
    return True
    sum = 0
    for elem in window:
        if elem:
            sum = sum + 1
    sz = np.size(window)
    return ((sum/sz) > 0.80)
        
def verificate_with_rcrd(seg_index, window_pos, window_width):
#     return True
    return np.any(rcrd_vectors[seg_index, window_pos:window_pos + window_width])

#### Probar con bandas de verde

In [ ]:

for seg_index in range(0,steps):
    segment = to_detect[offset_seg_height:offset_seg_height + seg_height,0:td_w]
    vector = calculate_row(segment, seg_height, td_w)
    

In [72]:
#FIP
offset_seg_height = 0

# from matplotlib import pyplot as plt
# x = np.arange(0,td_w) 
three_ch_copy = three_channel_edge.copy()

#La imagen la voy recorriendo de abajo hacia arriba
for seg_index in range(0, steps):
    #Calculo los vectores de cada segmento
    segment = to_detect[offset_seg_height:offset_seg_height + seg_height,0:td_w]
    vector = calculate_row(segment, seg_height, td_w)
    
    #Todo lo que no sea crop, es weed
    window_pos = 0;
    while (window_pos < td_w - window_width ):
        #Hay algo mal en el algoritmo
        if (is_crop(vector[window_pos:window_pos + window_width])):
            if (verificate_with_rcrd(seg_index, window_pos, window_width)):
            #Luego de verificar que sea crop, pinto ese sector
                sector = three_ch_copy[offset_seg_height:offset_seg_height + seg_height, window_pos: window_pos + window_width]
                sec_b, sec_g, sec_r = cv2.split(sector)
                sector = cv2.merge((sec_r, sec_g, sec_b)) #Lo pinto dando vuelta los canales
                three_ch_copy[offset_seg_height:offset_seg_height + seg_height, window_pos: window_pos + window_width] = sector
                window_pos = window_pos + window_width
            else:
                window_pos = window_pos + 1
        else:
            window_pos = window_pos + 1
    offset_seg_height = offset_seg_height + seg_height

In [73]:
mostrar_img('th', three_ch_copy)

### Region growing

In [68]:
#De manera recursiva repinta lo que encuentre a su alrededor
def paint(pixel):
    if (pixel[0]):
        aux = pixel[0]
        pixel [0] = pixel[2]
        pixel[2] = aux

def contagio(img, y_init, x_init):
    img_h, img_w, ch = img.shape
    x1= x_init-1
    x2= x_init+1
    y1= y_init-1
    y2= y_init+1
    if (x1 >= 0):
        paint(img[y_init,x1])
    if (x2 < img_w):
        paint(img[y_init,x2])
    if (y1 >= 0):
        paint(img[y1,x_init])
    if (y2 < img_h):
        paint(img[y2,x_init])

In [69]:
three_ch_copy2 = three_ch_copy.copy()
for y in range(0,td_h-1):
    for x in range(0,td_w-1):
        if (three_ch_copy2[y,x,2]): #BGR, el contagio es con R
            contagio(three_ch_copy2, y, x)
mostrar_imgs(['th contagio','th'], [three_ch_copy2, three_ch_copy])

In [ ]:
#Colorearlo en imagen de bordes (Poner valor 255 en segundo (crop) o tercer canal (weed)) 
#Sumar bordes clasificados con imagen 
result = cv2.add(original_td, three_ch_copy2)

In [ ]:
mostrar_img('resultado', result)

### Guardado de imágenes

In [ ]:
#Guardar resultado
file_name='total'
cv2.imwrite('img/row_test/' + file_name + '_resultado2.png', result)
# cv2.imwrite('img/row_test/' + file_name + '_with_lines.png', with_lines)